In [57]:
!pip install faiss-gpu
!pip install faiss-cpu

In [58]:
import numpy as np
from scipy import spatial
import faiss
from time import time
import matplotlib.pyplot as plt
from collections import defaultdict

## Helper Function

In [59]:
def semi_optimized_exhaustive_search(
        index_vectors: np.ndarray,
        query_vectors: np.ndarray,
        k: int,
):
    """
    This function performs an optimized exhaustive search.
    Args:
        index_vectors: An array of shape (n_index, dim) containing the index vectors.
        query_vectors: An array of shape (n_queries, dim) containing the query vectors.
        dim: The dimensionality of the vectors.
    Returns:
        An array of shape (n_queries, k) containing the indices of the k nearest neighbors for each query vector.
    """
    ann_lists = []
    for query_vec in query_vectors:
        distances = np.linalg.norm(index_vectors - query_vec, axis=1)
        ann_lists.append(list(np.argsort(distances)[:k]))
    return np.array(ann_lists)

In [60]:
def build_faiss_flatl2_index(
        index_vectors: np.ndarray,
        dim: int,
):
    """
    This function builds a Faiss flat L2 index.
    Args:
        index_vectors: An array of shape (n_index, dim) containing the index vectors.
        dim: The dimensionality of the vectors.
    Returns:
        A Faiss flat L2 index.
    """
    index = faiss.IndexFlatL2(dim)
    index.add(index_vectors)
    return index

In [61]:
def faiss_search(
        query_vectors: np.ndarray,
        index: faiss.Index,
        k: int,
):
    """
    This function uses a Faiss index to search for the k-nearest neighbors of query_vectors.
    Args:
        query_vectors: An array of shape (n_queries, dim) containing the query vectors.
        index: A Faiss index.
        k: The number of nearest neighbors to retrieve.
    Returns:
        An array of shape (, ) containing the indices of the k-nearest neighbors for each query vector.
    """
    distances, indices = index.search(query_vectors, k)
    return indices

In [62]:
def build_faiss_lsh_index(
        index_vectors: np.ndarray,
        dim: int,
        nbits: int,
):
    """
    This function builds a Faiss LSH index.
    Args:
        index_vectors: An array of shape (n_index, dim) containing the index vectors.
        dim: The dimensionality of the vectors.
        nbits: The number of bits to use in the hash.
    Returns:
        A Faiss LSH index.
    """
    index = faiss.IndexLSH(dim, nbits)
    index.add(index_vectors)
    return index

In [63]:
def compute_recall_at_k(
        nn_gt: np.ndarray,
        ann: np.ndarray,
        k: int,
):
    """
    This function computes the recall@k.
    Args:
        nn_gt: The ground truth nearest neighbors.
        ann: The approximate nearest neighbors.
        k: The number of nearest neighbors to consider.
    Returns:
        The recall@k.
    """
    return round(sum([len(set(ann[i]) & set(nn_gt[i])) / k for i in range(len(ann))])/len(ann), 3)

# 2.1 -- LSH vs Naive Exhaustive Search (Regular Index Vectors)
### You just have to run the following cells and add the following results to the report:
* running time of the ground truth computation with semi_optimized_exhaustive_search (wall time)
* running time of creating faiss_lsh_index (wall time)
* running time of faiss_search over query_vectors with faiss_lsh_index (wall time)
* recall@10 for faiss_lsh_ann

In [64]:
query_vectors = np.load('data/query_vectors.npy')
index_vectors = np.load('data/index_vectors.npy')
k=10
dim = index_vectors.shape[1]

In [65]:
%%time
gt_nn = semi_optimized_exhaustive_search(index_vectors, query_vectors, k)

CPU times: user 5.7 s, sys: 29.8 ms, total: 5.72 s
Wall time: 6.24 s


In [66]:
%%time
faiss_lsh_index = build_faiss_lsh_index(index_vectors, dim, nbits=2000)

CPU times: user 736 ms, sys: 60.7 ms, total: 797 ms
Wall time: 918 ms


In [67]:
%%time
faiss_lsh_ann = faiss_search(query_vectors, faiss_lsh_index, k)

CPU times: user 772 ms, sys: 1.71 ms, total: 773 ms
Wall time: 434 ms


In [68]:
print(f"recall@10 for faiss_lsh_index: {compute_recall_at_k(gt_nn, faiss_lsh_ann, k)}")

recall@10 for faiss_lsh_index: 0.138


# 2.2 -- Custom Indexing Algorithm
Build an indexing algorithm that satisfies the following requirements:
* The indexing algorithm should be able to handle vectors of different dimensions
* The running time of the indexing should be less than half of the running time of semi_optimized_exhaustive_search), reported in Section 2.1.
* The running time of searching over the index should be less than a third (1/3) of the time of the semi_optimized_exhaustive_search function, reported in Section 2.1.
* The performance (in terms of recall@10) of the indexing algorithm should be at least 0.8.

The last three bullets should also appear in the report.
You are allowed to add as many helper functions as you need. You cannot use faiss of scipy libraries for this task. Numpy is allowed.

You can also test your algorithm with the additional two query-index sets by replacing the calls made few cells ago to:

    query_vectors = np.load('data/query_vectors2.npy')
    index_vectors = np.load('data/index_vectors2.npy')
or:

    query_vectors = np.load('data/query_vectors3.npy')
    index_vectors = np.load('data/index_vectors3.npy')
    
the aforementioned requirements should also be satisfied over these two query-index sets. No need to insert the results over these two to the report.

In [69]:
from sklearn.cluster import KMeans

In [70]:
class SNNIndex:
    """
    'Soft Kmeans Indexing' - Indexing class for vectorDB
    """
    def __init__(self, noc, div_ratio):
        """
        :param noc: Number of Clusters (Kmeans parameter).
        :param div_ratio: Division Ratio - determined the number of neighbored clusters.
        """
        self.noc = noc
        self.div_ratio = div_ratio
        self.non = int(self.noc / self.div_ratio)  # Number of Neighbors
        self.indexing = {ci: [] for ci in range(noc)}  # dictionary of (label: vectors)
        self.centroids = None  # centroids list (vectors)
        self.vec2ind = {}  # dictionary of (vector: index)

    def index(self, vectors):
        """
        This function builds an index from scratch.
        :param vectors: An array of vectors.
        """
        # perform Kmeans classification
        self.centroids, labels = SKMIndex.perform_kmeans(self.noc)

        # find closest neighbors of each cluster
        centroids_neighbors = {ci: [] for ci in range(self.noc)}
        for label, cent in enumerate(self.centroids):
            dist_from_rest = np.linalg.norm(self.centroids - cent, axis=1)
            centroids_neighbors[label] = list(np.argsort(dist_from_rest)[1:self.non + 1])

        # fill dictionary of (label: vectors) - each key label contains vectors from same cluster and neighbor clusters
        for label, vector in zip(labels, vectors):
            self.indexing[label].append(vector)
            for neighbor_label in centroids_neighbors[label]:
                self.indexing[neighbor_label].append(vector)
        for key in self.indexing.keys():
            self.indexing[key] = np.array(self.indexing[key])

        # fill dictionary of (vector: index) - each key vector contains the vector index in 'index_vectors'
        for ind, vector in enumerate(vectors):
            self.vec2ind[tuple(vector)] = ind


    def search(self, queries, k):
        """
        This function search the most similar items for each query
        :param queries: An array of vectors.
        :param k: The number of nearest neighbors to retrieve.
        :return: list of lists
        """
        search_results = []
        for qv in queries:
            # find the nearest centroid to the 'qv' vector (the index)
            dists_from_centroids = np.linalg.norm(self.centroids - qv, axis=1)
            nearest_centroid = np.argmin(dists_from_centroids)  # 1
            # find distances between the 'qv' vector to the corresponding vectors belong to the index
            dists = np.linalg.norm(self.indexing[nearest_centroid] - qv, axis=1)
            # sort and save the closest 'k' vectors
            sorted_dists_ind = list(np.argsort(dists)[:k])
            results = [self.indexing[nearest_centroid][i] for i in sorted_dists_ind]
            search_results.append([self.vec2ind[tuple(vector)] for vector in results])
        return search_results


    @staticmethod
    def perform_kmeans(k, random_state=42, n_init=1):
        """
        This function run the Kmeans algorithm
        Args:
            k: number of clusters
            random_state: constant intialization
            n_init: number of different centorids initalizations
        Returns:
            centroids of each cluster (list), labels for each vector (list)
        """
        kmeans = KMeans(n_clusters=k, random_state=random_state, n_init=n_init)
        kmeans.fit(index_vectors)
        labels = kmeans.predict(index_vectors)
        centroids = kmeans.cluster_centers_
        return centroids, labels

In [71]:
#TODO: Write your code for 2.2.2 here
# You are allowed to add more arguments to the functions and create more functions if needed.

def custom_indexing_algorithm(index_vectors, dim, noc, div_ratio):
    """
    This function builds an index from scratch.
    Args:
        index_vectors: An array of shape (n_index, dim) containing the index vectors.
        dim: The dimensionality of the vectors.
        noc: Number of Clusters (Kmeans parameter).
        div_ratio: Division Ratio - determined the number of neighbors clusters.
    Returns:
        An index.
    """
    skm_index = SNNIndex(noc=50, div_ratio=8)
    skm_index.index(index_vectors)
    return skm_index


def custom_index_search(query_vectors, index, k):
    """
    This function searches over the custom index.
    Args:
        query_vectors: An array of shape (n_queries, dim) containing the query vectors.
        index: The custom index.
        k: The number of nearest neighbors to retrieve.
    """
    res = index.search(query_vectors, k)
    return res


In [72]:
# Add hyperparameters here (if needed)
noc = 50
div_ratio = 8

In [73]:
%%time
custom_index = custom_indexing_algorithm(index_vectors, dim, noc, div_ratio)

CPU times: user 1.41 s, sys: 145 ms, total: 1.55 s
Wall time: 1.2 s


In [74]:
%%time
custom_index_ann = custom_index_search(query_vectors, custom_index, k)

CPU times: user 1.36 s, sys: 15.1 ms, total: 1.37 s
Wall time: 2.88 s


In [75]:
print(f"recall@10 for custom_index_search: {compute_recall_at_k(gt_nn, custom_index_ann, k)}")

recall@10 for custom_index_search: 0.937


In [76]:
# results for all data
gt_time = []
indexing_time = []
searching_time = []
accuracy = []

for suf in ['', '2', '3']:
  query_vectors = np.load(f'data/query_vectors{suf}.npy')
  index_vectors = np.load(f'data/index_vectors{suf}.npy')
  k=10

  # obtain ground truth
  st = time()
  gt_nn = semi_optimized_exhaustive_search(index_vectors, query_vectors, k)
  gt_time.append(time() - st)

  # use custom index

  noc, div_ratio = 50, 80

  st = time()
  custom_index = custom_indexing_algorithm(index_vectors, dim, noc, div_ratio)
  indexing_time.append(time() - st)

  st = time()
  custom_index_ann = custom_index_search(query_vectors, custom_index, k)
  searching_time.append(time() - st)

  acc = compute_recall_at_k(gt_nn, custom_index_ann, k)
  accuracy.append(acc)



In [77]:
print("ground truth time: ", gt_time)
print("custom index indexing time: ", indexing_time)
print("ustom index searching time: ", searching_time)
print("accuracy: ", accuracy)

ground truth time:  [8.196296691894531, 14.302134275436401, 5.569471836090088]
custom index indexing time:  [2.553548812866211, 1.4656586647033691, 0.9073696136474609]
ustom index searching time:  [1.6720178127288818, 1.2402920722961426, 0.7980630397796631]
accuracy:  [0.937, 0.912, 0.96]
